<center>
<img src="../../../mlcourse.ai/img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>
Автор материала: Юрий Кашницкий, программист-исследователь Mail.Ru Group <br> 

Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 6 (демо). Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина</center>
<img src='../../../mlcourse.ai/img/wine_quality.jpg' width=30%>

**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1gsNxgkd0VqidZp4lh9mnCQnJw3b0IFR1C4WBES86J40).**

In [151]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [2]:
data = pd.read_csv('../../../mlcourse.ai/data/winequality-white.csv', sep=';')

In [3]:
data.sample(10).T

,2894,1201,965,2504,2983,3345,896,12,1125,4249
fixed acidity,6.40000,7.0000,8.3000,8.50000,6.00000,6.8000,6.8000,7.900,6.3000,5.50000
volatile acidity,0.12500,0.1700,0.2300,0.30000,0.26000,0.1800,0.1800,0.180,0.5500,0.23000
citric acid,0.36000,0.3700,0.4300,0.28000,0.32000,0.2400,0.2800,0.370,0.4500,0.19000
residual sugar,1.40000,1.5000,3.2000,3.10000,3.80000,9.8000,8.7000,1.200,13.0000,2.20000
chlorides,0.04400,0.0280,0.0350,0.05400,0.02900,0.0580,0.0470,0.040,0.0470,0.04400
free sulfur dioxide,22.00000,26.0000,14.0000,54.00000,48.00000,64.0000,52.0000,16.000,33.0000,39.00000
total sulfur dioxide,68.00000,75.0000,101.0000,174.00000,180.00000,188.0000,242.0000,75.000,182.0000,161.00000
density,0.99014,0.9922,0.9928,0.99543,0.99011,0.9952,0.9952,0.992,0.9974,0.99209
pH,3.15000,3.3000,3.1500,3.21000,3.15000,3.1300,3.2200,3.180,3.2000,3.19000
sulphates,0.50000,0.4600,0.3600,0.43000,0.34000,0.5100,0.5300,0.630,0.4600,0.43000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [113]:
y = data.quality
X = data.drop('quality', axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

In [95]:
X.shape, y.shape, X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((4898, 11), (4898,), (3428, 11), (1470, 11), (3428,), (1470,))

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [96]:
linreg = LinearRegression(n_jobs=8)
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=8, normalize=False)

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [8]:
print("Mean squared error (train): %.3f" % mse(y_train, linreg.predict(X_train)))
print("Mean squared error (test): %.3f" % mse(y_valid, linreg.predict(X_valid)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**  
**<i> 7 </i>**

In [33]:
for i, col in enumerate(X_train.columns):
    print(i, col)

0 fixed acidity
1 volatile acidity
2 citric acid
3 residual sugar
4 chlorides
5 free sulfur dioxide
6 total sulfur dioxide
7 density
8 pH
9 sulphates
10 alcohol


In [45]:
linreg_coef = pd.DataFrame(linreg.coef_)
linreg_coef.sort_values(by=0).T

,7,1,2,6,5,10,3,0,4,9,8
0,-227.551202,-1.886874,-0.001485,0.000333,0.002439,0.105264,0.108183,0.113731,0.352051,0.552495,0.99583


## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [97]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**  
<i>2 - citric acid</i>

In [98]:
lasso1_coef = pd.DataFrame(lasso1.coef_)
lasso1_coef.sort_values(by=0).T

,1,0,6,2,4,7,8,9,5,3,10
0,-0.967909,-0.05977,-0.001108,0.0,-0.0,-0.0,0.0,0.0,0.005223,0.022231,0.352083


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [99]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [100]:
lasso_cv.alpha_ # 0.000001

1e-06

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**  
2 - citric acid

In [101]:
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_)
lasso_cv_coef.sort_values(by=0).T

,7,1,2,6,5,3,0,10,4,9,8
0,-223.055887,-1.888451,-0.002444,0.000314,0.002466,0.106579,0.110345,0.11044,0.331071,0.546349,0.981016


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [102]:
print("Mean squared error (train): %.3f" % mse(y_train, lasso1.predict(X_train)))
print("Mean squared error (test): %.3f" % mse(y_valid, lasso1.predict(X_valid)))

Mean squared error (train): 0.583
Mean squared error (test): 0.588


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [105]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [125]:
X.sample()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
4679,4.7,0.335,0.14,1.3,0.036,69.0,168.0,0.99212,3.47,0.46,10.5


In [144]:
rf_train_pred = forest.predict(X_train)
rf_cv = cross_val_score(forest, X_train, y_train, scoring='neg_mean_squared_error')
rf_test_pred = forest.predict(X_valid)

In [163]:
print("Mean squared error (train): %.3f" % mse(y_train, rf_train_pred))
print("Mean squared error (cv): %.3f" % rf_cv.mean())
print("Mean squared error (test): %.3f" % mse(y_valid, rf_test_pred))

Mean squared error (train): 0.076
Mean squared error (cv): -0.462
Mean squared error (test): 0.422


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [165]:
%time
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, forest_params)
locally_best_forest.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7], 'max_features': [6, 7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [166]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 19, 'max_features': 6, 'min_samples_leaf': 1},
 0.42144574416375985)

In [167]:
%time
locally_best_forest_train_pred = locally_best_forest.predict(X_train)
locally_best_forest_cv = cross_val_score(locally_best_forest, X_train, y_train, scoring='neg_mean_squared_error')
locally_best_forest_test_pred = locally_best_forest.predict(X_valid)

In [168]:
print("Mean squared error (train): %.3f" % mse(y_train, rf_train_pred))
print("Mean squared error (cv): %.3f" % rf_cv.mean())
print("Mean squared error (test): %.3f" % mse(y_valid, rf_test_pred))

Mean squared error (train): 0.076
Mean squared error (cv): -0.462
Mean squared error (test): 0.422


**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [186]:
%time
forest_ = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state=17)
forest_.fit(X_train, y_train)

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 15.5 µs


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=None, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [204]:
%time
forest_train_pred_ = forest_.predict(X_train)
forest_cv_ = cross_val_score(forest_, X_train, y_train, scoring='neg_mean_squared_error')
forest_test_pred_ = forest_.predict(X_valid)

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 16.2 µs


In [205]:
print("Mean squared error (train): %.3f" % mse(y_train, forest_train_pred_))
print("Mean squared error (cv): %.3f" % forest_cv_.mean())
print("Mean squared error (test): %.3f" % mse(y_valid, forest_test_pred))

Mean squared error (train): 0.084
Mean squared error (cv): -0.456
Mean squared error (test): 0.407


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

10 - alco

In [198]:
rf_importance = pd.DataFrame(forest_.feature_importances_)
rf_importance.sort_values(by=0).T

,9,2,0,4,7,3,6,8,5,1,10
0,0.061184,0.062945,0.064268,0.067982,0.069367,0.07016,0.071318,0.072806,0.116147,0.119393,0.224432


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**